In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import time
import os
import json

class NSEIndexAnalyzer:
    def __init__(self):
        self.base_dir = 'index_analysis'
        self.results_dir = os.path.join(self.base_dir, 'results')
        os.makedirs(self.results_dir, exist_ok=True)
        
        # List of Indian indices to analyze with their Yahoo Finance tickers
        self.equity_market_indices = [
            {
                'name': 'BSE SENSEX',
                'ticker': '^BSESN'
            },
            {
                'name': 'NIFTY 50',
                'ticker': '^NSEI'
            },
            {
                'name': 'NIFTY BANK',
                'ticker': '^NSEBANK'
            },
            {
                'name': 'NIFTY IT',
                'ticker': '^CNXIT'
            },
            {
                'name': 'NIFTY PHARMA',    #only 1 day data available
                'ticker': '^CNXPHARMA'
            },
            {
                'name': 'NIFTY AUTO',
                'ticker': '^CNXAUTO'
            },
            {
                'name': 'NIFTY FINANCIAL SERVICES',
                'ticker': 'NIFTY_FIN_SERVICE.NS'
            },
            {
                'name': 'NIFTY FMCG',
                'ticker': '^CNXFMCG'
            },
            {
                'name': 'NIFTY METAL',
                'ticker': '^CNXMETAL'
            },
            {
                'name': 'NIFTY REALTY',
                'ticker': '^CNXREALTY'
            },
            {
                'name': 'NIFTY MEDIA',
                'ticker': '^CNXMEDIA'
            },
            {
                'name': 'NIFTY PSU BANK',
                'ticker': '^CNXPSUBANK'
            },
            # {
            #     'name': 'NIFTY MIDCAP 50',    No ticker available in yfinance for NIFTY MIDCAP 50
            #     'ticker': '^NIFMDCP50'
            # },
            # {
            #     'name': 'NIFTY SMALLCAP 50',    No ticker available in yfinance for NIFTY SMALLCAP 50
            #     'ticker': '^NIFSMCP50'
            # },
            # {
            #     'name': 'NIFTY PVT BANK',
            #     'ticker': 'NIFTYPVTBANK.NS'    No ticker available in yfinance for NIFTY PVT BANK
            # },
            {
                'name': 'NIFTY PSE',
                'ticker': '^CNXPSE'
            },
            {
                'name': 'NIFTY ENERGY',
                'ticker': '^CNXENERGY'
            },
            {
                'name': 'NIFTY MNC',
                'ticker': '^CNXMNC'
            },
            {
                'name': 'NIFTY INFRA',
                'ticker': '^CNXINFRA'
            }
        ]

    def calculate_rsi(self, prices_tuple, periods=14):
        """RSI calculation method"""
        try:
            prices = pd.Series(prices_tuple)
            
            if len(prices) < periods + 1:
                return None
                
            # Calculate price changes
            delta = prices.diff()
            
            # Split gains and losses
            gains = delta.copy()
            losses = delta.copy()
            
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calculate initial averages
            first_avg_gain = gains[1:periods+1].mean()
            first_avg_loss = losses[1:periods+1].mean()
            
            avg_gains = [first_avg_gain]
            avg_losses = [first_avg_loss]
            
            # Calculate subsequent values
            for i in range(periods+1, len(gains)):
                avg_gain = (avg_gains[-1] * (periods-1) + gains[i]) / periods
                avg_loss = (avg_losses[-1] * (periods-1) + losses[i]) / periods
                avg_gains.append(avg_gain)
                avg_losses.append(avg_loss)
            
            # Calculate final RSI
            if avg_losses[-1] == 0:
                rsi = 100.0
            else:
                rs = avg_gains[-1] / avg_losses[-1]
                rsi = 100 - (100 / (1 + rs))
            
            return float(rsi)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def calculate_tema(self, prices, period=50):
        """
        Calculate Triple Exponential Moving Average (TEMA)
        
        Args:
            prices (pd.Series): Price series
            period (int): Period for TEMA calculation
        
        Returns:
            float: TEMA value
        """
        try:
            if len(prices) < period:
                return None
            
            # Calculate EMAs
            ema1 = prices.ewm(span=period, adjust=False).mean()
            ema2 = ema1.ewm(span=period, adjust=False).mean()
            ema3 = ema2.ewm(span=period, adjust=False).mean()
            
            # TEMA Formula: TEMA = 3 * EMA1 - 3 * EMA2 + EMA3
            tema = 3 * ema1.iloc[-1] - 3 * ema2.iloc[-1] + ema3.iloc[-1]
            
            return float(tema)
        
        except Exception as e:
            print(f"Error calculating TEMA: {e}")
            return None

    def get_index_data(self, index_info):
        """Fetch detailed index data"""
        try:
            time.sleep(1)  # Rate limiting
            ticker_symbol = index_info['ticker']
            ticker = yf.Ticker(ticker_symbol)
            
            # Get historical data for 1 year
            hist_data = ticker.history(period="1y", interval="1d")

            # Check if 1 year data is available, else get maximum available data
            if hist_data.empty:
                hist_data = ticker.history(period="max", interval="1d")
                if hist_data.empty:
                    print(f"No data available for {index_info['name']} ({ticker_symbol})")
                    return None

            # Calculate basic metrics
            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            
            # Calculate indicators
            latest_rsi = self.calculate_rsi(prices_tuple)
            
            # Calculate EMA-50 and TEMA-50
            prices_series = hist_data['Close']
            ema_50 = prices_series.ewm(span=50, adjust=False).mean().iloc[-1]
            tema_50 = self.calculate_tema(prices_series)
            
            # Calculate 20-day and 50-day simple moving averages
            sma_20 = hist_data['Close'].rolling(window=20).mean().iloc[-1] if len(hist_data) >= 20 else None
            sma_50 = hist_data['Close'].rolling(window=50).mean().iloc[-1] if len(hist_data) >= 50 else None
            
            # Calculate daily returns
            daily_returns = hist_data['Close'].pct_change()
            
            # Calculate volatility (20-day standard deviation of returns)
            volatility_20 = daily_returns.rolling(window=20).std().iloc[-1] * (252 ** 0.5)  # Annualized
            
            # Calculate 52-week high and low
            high_52w = hist_data['High'].max()
            low_52w = hist_data['Low'].min()
            
            # Calculate percent from 52-week high
            pct_from_high = (latest_price - high_52w) / high_52w * 100
            
            # Prepare index information dictionary
            index_data = {
                'Index_Name': index_info['name'],
                'Ticker': ticker_symbol,
                'Latest_Price': round(latest_price, 2),
                'RSI_14': round(latest_rsi, 2) if latest_rsi is not None else 'N/A',
                'EMA_50': round(ema_50, 2) if ema_50 is not None else 'N/A',
                'TEMA_50': round(tema_50, 2) if tema_50 is not None else 'N/A',
                'TEMA_50-EMA_50': round(tema_50 - ema_50, 2) if tema_50 is not None and ema_50 is not None else 'N/A',
                'SMA_20': round(sma_20, 2) if sma_20 is not None else 'N/A',
                'SMA_50': round(sma_50, 2) if sma_50 is not None else 'N/A',
                'Volatility_20D': round(volatility_20 * 100, 2) if not pd.isna(volatility_20) else 'N/A',
                '52W_High': round(high_52w, 2),
                '52W_Low': round(low_52w, 2),
                'Pct_From_52W_High': round(pct_from_high, 2),
                'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 'N/A',
                'Daily_Return': round(daily_returns.iloc[-1] * 100, 2) if not pd.isna(daily_returns.iloc[-1]) else 'N/A'
            }
            
            return index_data

        except Exception as e:
            print(f"Error processing {index_info['name']} ({index_info['ticker']}): {e}")
            return None

    def analyze_indices(self):
        """Analyze all indices and generate Excel report"""
        # Timestamp for unique filename
        current_date = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_file = os.path.join(self.results_dir, f'indian_indices_analysis_{current_date}.xlsx')
        
        # Process all indices
        results = []
        for index_info in self.equity_market_indices:
            print(f"Processing {index_info['name']} ({index_info['ticker']})...")
            index_data = self.get_index_data(index_info)
            if index_data:
                results.append(index_data)
        
        # Convert to DataFrame
        if results:
            df = pd.DataFrame(results)
            
            # Create Excel writer
            with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                # Write main index data
                df.to_excel(writer, sheet_name='Indian_Indices', index=False)
                
                # Get workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets['Indian_Indices']
                
                # Add formats for conditional formatting
                green_format = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
                red_format = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
                
                # Apply conditional formatting to RSI column
                rsi_col = df.columns.get_loc('RSI_14') + 1  # +1 for Excel's 1-indexing
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'less than', 'value': 30, 'format': green_format})
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'greater than', 'value': 70, 'format': red_format})
            
            print(f"\nAnalysis complete. Results saved to {excel_file}")
        else:
            print("No index data was successfully retrieved.")

if __name__ == "__main__":
    analyzer = NSEIndexAnalyzer()
    analyzer.analyze_indices()

Processing NIFTY 50 (^NSEI)...
Processing NIFTY BANK (^NSEBANK)...
Processing NIFTY IT (^CNXIT)...
Processing NIFTY PHARMA (^CNXPHARMA)...
Processing NIFTY AUTO (^CNXAUTO)...
Processing NIFTY FINANCIAL SERVICES (NIFTY_FIN_SERVICE.NS)...
Processing NIFTY FMCG (^CNXFMCG)...
Processing NIFTY METAL (^CNXMETAL)...
Processing NIFTY REALTY (^CNXREALTY)...
Processing NIFTY MEDIA (^CNXMEDIA)...
Processing NIFTY PSU BANK (^CNXPSUBANK)...
Processing NIFTY MIDCAP 50 (^NIFMDCP50)...


$^NIFMDCP50: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$^NIFMDCP50: possibly delisted; no timezone found


No data available for NIFTY MIDCAP 50 (^NIFMDCP50)
Processing NIFTY SMALLCAP 50 (^NIFSMCP50)...


$^NIFSMCP50: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$^NIFSMCP50: possibly delisted; no timezone found


No data available for NIFTY SMALLCAP 50 (^NIFSMCP50)
Processing NIFTY PSE (^CNXPSE)...
Processing NIFTY ENERGY (^CNXENERGY)...
Processing NIFTY MNC (^CNXMNC)...
Processing NIFTY INFRA (^CNXINFRA)...

Analysis complete. Results saved to index_analysis\results\indian_indices_analysis_20250227_164916.xlsx


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import time
import os
import json
import random

class NSEIndexAnalyzer:
    def __init__(self):
        self.base_dir = 'index_analysis'
        self.results_dir = os.path.join(self.base_dir, 'results')
        os.makedirs(self.results_dir, exist_ok=True)
        
        # List of Indian indices to analyze with their Yahoo Finance tickers
        self.equity_market_indices = [
            {
                'name': 'NIFTY 50',
                'ticker': '^NSEI'
            },
            {
                'name': 'BSE SENSEX',
                'ticker': '^BSESN'
            },
            {
                'name': 'NIFTY BANK',
                'ticker': '^NSEBANK'
            },
            {
                'name': 'NIFTY IT',
                'ticker': '^CNXIT'
            },
            {
                'name': 'NIFTY PHARMA',
                'ticker': '^CNXPHARMA'
            },
            {
                'name': 'NIFTY AUTO',
                'ticker': '^CNXAUTO'
            },
            {
                'name': 'NIFTY FIN SERVICES',
                'ticker': '^CNXFIN'
            },
            {
                'name': 'NIFTY FMCG',
                'ticker': '^CNXFMCG'
            },
            {
                'name': 'NIFTY METAL',
                'ticker': '^CNXMETAL'
            },
            {
                'name': 'NIFTY REALTY',
                'ticker': '^CNXREALTY'
            },
            {
                'name': 'NIFTY MEDIA',
                'ticker': '^CNXMEDIA'
            },
            {
                'name': 'NIFTY PSU BANK',
                'ticker': '^CNXPSUBANK'
            },
            {
                'name': 'NIFTY MIDCAP 50',
                'ticker': '^NSEMDCP50'
            },
            {
                'name': 'NIFTY SMALLCAP 50',
                'ticker': 'NIFTYSMLCAP50.NS'
            },
            # {
            #     'name': 'NIFTY PVT BANK',
            #     'ticker': 'NIFTYPVTBANK.NS'    No ticker available in yfinance for NIFTY PVT BANK
            # },
            {
                'name': 'NIFTY PSE',
                'ticker': '^CNXPSE'
            },
            {
                'name': 'NIFTY ENERGY',
                'ticker': '^CNXENERGY'
            },
            {
                'name': 'NIFTY MNC',
                'ticker': '^CNXMNC'
            },
            {
                'name': 'NIFTY INFRA',
                'ticker': '^CNXINFRA'
            }
        ]

    def calculate_rsi(self, prices_tuple, periods=14):
        """RSI calculation method"""
        try:
            prices = pd.Series(prices_tuple)
            
            if len(prices) < periods + 1:
                return None
                
            # Calculate price changes
            delta = prices.diff()
            
            # Split gains and losses
            gains = delta.copy()
            losses = delta.copy()
            
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calculate initial averages
            first_avg_gain = gains[1:periods+1].mean()
            first_avg_loss = losses[1:periods+1].mean()
            
            avg_gains = [first_avg_gain]
            avg_losses = [first_avg_loss]
            
            # Calculate subsequent values
            for i in range(periods+1, len(gains)):
                avg_gain = (avg_gains[-1] * (periods-1) + gains[i]) / periods
                avg_loss = (avg_losses[-1] * (periods-1) + losses[i]) / periods
                avg_gains.append(avg_gain)
                avg_losses.append(avg_loss)
            
            # Calculate final RSI
            if avg_losses[-1] == 0:
                rsi = 100.0
            else:
                rs = avg_gains[-1] / avg_losses[-1]
                rsi = 100 - (100 / (1 + rs))
            
            return float(rsi)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def calculate_tema(self, prices, period=50):
        """
        Calculate Triple Exponential Moving Average (TEMA)
        
        Args:
            prices (pd.Series): Price series
            period (int): Period for TEMA calculation
        
        Returns:
            float: TEMA value
        """
        try:
            if len(prices) < period:
                return None
            
            # Calculate EMAs
            ema1 = prices.ewm(span=period, adjust=False).mean()
            ema2 = ema1.ewm(span=period, adjust=False).mean()
            ema3 = ema2.ewm(span=period, adjust=False).mean()
            
            # TEMA Formula: TEMA = 3 * EMA1 - 3 * EMA2 + EMA3
            tema = 3 * ema1.iloc[-1] - 3 * ema2.iloc[-1] + ema3.iloc[-1]
            
            return float(tema)
        
        except Exception as e:
            print(f"Error calculating TEMA: {e}")
            return None

    def get_index_data(self, index_info, max_retries=3):
        """Fetch detailed index data with retry logic"""
        for attempt in range(max_retries):
            try:
                # Add randomized delay between 3-6 seconds to avoid rate limiting
                delay = 3 + random.random() * 3
                print(f"Waiting {delay:.1f} seconds before requesting data...")
                time.sleep(delay)
                
                ticker_symbol = index_info['ticker']
                ticker = yf.Ticker(ticker_symbol)
                
                # Get historical data for 1 year
                hist_data = ticker.history(period="1y", interval="1d")

                # Check if 1 year data is available, else get maximum available data
                if hist_data.empty:
                    hist_data = ticker.history(period="max", interval="1d")
                    if hist_data.empty:
                        print(f"No data available for {index_info['name']} ({ticker_symbol})")
                        return None

                # Calculate basic metrics
                latest_price = hist_data['Close'].iloc[-1]
                prices_tuple = tuple(hist_data['Close'].values)
                
                # Calculate indicators
                latest_rsi = self.calculate_rsi(prices_tuple)
                
                # Calculate EMA-50 and TEMA-50
                prices_series = hist_data['Close']
                ema_50 = prices_series.ewm(span=50, adjust=False).mean().iloc[-1]
                tema_50 = self.calculate_tema(prices_series)
                
                # Calculate 20-day and 50-day simple moving averages
                sma_20 = hist_data['Close'].rolling(window=20).mean().iloc[-1] if len(hist_data) >= 20 else None
                sma_50 = hist_data['Close'].rolling(window=50).mean().iloc[-1] if len(hist_data) >= 50 else None
                
                # Calculate daily returns
                daily_returns = hist_data['Close'].pct_change()
                
                # Calculate volatility (20-day standard deviation of returns)
                volatility_20 = daily_returns.rolling(window=20).std().iloc[-1] * (252 ** 0.5)  # Annualized
                
                # Calculate 52-week high and low
                high_52w = hist_data['High'].max()
                low_52w = hist_data['Low'].min()
                
                # Calculate percent from 52-week high
                pct_from_high = (latest_price - high_52w) / high_52w * 100
                
                # Prepare index information dictionary
                index_data = {
                    'Index_Name': index_info['name'],
                    'Ticker': ticker_symbol,
                    'Latest_Price': round(latest_price, 2),
                    'RSI_14': round(latest_rsi, 2) if latest_rsi is not None else 'N/A',
                    'EMA_50': round(ema_50, 2) if ema_50 is not None else 'N/A',
                    'TEMA_50': round(tema_50, 2) if tema_50 is not None else 'N/A',
                    'TEMA_50-EMA_50': round(tema_50 - ema_50, 2) if tema_50 is not None and ema_50 is not None else 'N/A',
                    'SMA_20': round(sma_20, 2) if sma_20 is not None else 'N/A',
                    'SMA_50': round(sma_50, 2) if sma_50 is not None else 'N/A',
                    'Volatility_20D': round(volatility_20 * 100, 2) if not pd.isna(volatility_20) else 'N/A',
                    '52W_High': round(high_52w, 2),
                    '52W_Low': round(low_52w, 2),
                    'Pct_From_52W_High': round(pct_from_high, 2),
                    'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 'N/A',
                    'Daily_Return': round(daily_returns.iloc[-1] * 100, 2) if not pd.isna(daily_returns.iloc[-1]) else 'N/A'
                }
                
                return index_data

            except Exception as e:
                error_msg = str(e)
                print(f"Attempt {attempt+1}/{max_retries} - Error processing {index_info['name']} ({index_info['ticker']}): {error_msg}")
                
                if "Too Many Requests" in error_msg or "Rate limited" in error_msg:
                    # Exponential backoff - wait longer with each retry
                    wait_time = (2 ** attempt) * 10 + random.random() * 5
                    print(f"Rate limited. Waiting {wait_time:.1f} seconds before retrying...")
                    time.sleep(wait_time)
                else:
                    # For other errors, wait a shorter time
                    time.sleep(2)
        
        print(f"Failed to retrieve data for {index_info['name']} after {max_retries} attempts")
        return None

    def analyze_indices(self):
        """Analyze all indices and generate Excel report"""
        # Timestamp for unique filename
        current_date = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_file = os.path.join(self.results_dir, f'indian_indices_analysis_{current_date}.xlsx')
        
        # Process all indices
        results = []
        for index_info in self.equity_market_indices:
            print(f"Processing {index_info['name']} ({index_info['ticker']})...")
            index_data = self.get_index_data(index_info)
            if index_data:
                results.append(index_data)
        
        # Convert to DataFrame
        if results:
            df = pd.DataFrame(results)
            
            # Create Excel writer
            with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                # Write main index data
                df.to_excel(writer, sheet_name='Indian_Indices', index=False)
                
                # Get workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets['Indian_Indices']
                
                # Add formats for conditional formatting
                green_format = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
                red_format = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
                
                # Apply conditional formatting to RSI column
                rsi_col = df.columns.get_loc('RSI_14') + 1  # +1 for Excel's 1-indexing
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'less than', 'value': 30, 'format': green_format})
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'greater than', 'value': 70, 'format': red_format})
            
            print(f"\nAnalysis complete. Results saved to {excel_file}")
        else:
            print("No index data was successfully retrieved.")

if __name__ == "__main__":
    analyzer = NSEIndexAnalyzer()
    analyzer.analyze_indices()

Processing NIFTY 50 (^NSEI)...
Waiting 3.7 seconds before requesting data...
Processing BSE SENSEX (^BSESN)...
Waiting 3.9 seconds before requesting data...
Processing NIFTY BANK (^NSEBANK)...
Waiting 5.0 seconds before requesting data...
Processing NIFTY IT (^CNXIT)...
Waiting 5.2 seconds before requesting data...
Processing NIFTY PHARMA (^CNXPHARMA)...
Waiting 5.9 seconds before requesting data...
Processing NIFTY AUTO (^CNXAUTO)...
Waiting 4.7 seconds before requesting data...
Processing NIFTY FIN SERVICES (^CNXFIN)...
Waiting 5.8 seconds before requesting data...
Processing NIFTY FMCG (^CNXFMCG)...
Waiting 3.2 seconds before requesting data...
Processing NIFTY METAL (^CNXMETAL)...
Waiting 5.5 seconds before requesting data...
Processing NIFTY REALTY (^CNXREALTY)...
Waiting 3.4 seconds before requesting data...
Processing NIFTY MEDIA (^CNXMEDIA)...
Waiting 4.6 seconds before requesting data...
Processing NIFTY PSU BANK (^CNXPSUBANK)...
Waiting 6.0 seconds before requesting data...
P

$^NIFTYSMLCAP50: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$^NIFTYSMLCAP50: possibly delisted; no timezone found


No data available for NIFTY SMALLCAP 50 (^NIFTYSMLCAP50)
Processing NIFTY PVT BANK (NIFTYPVTBANK.NS)...
Waiting 4.3 seconds before requesting data...


$NIFTYPVTBANK.NS: possibly delisted; no price data found  (period=1y)
NIFTYPVTBANK.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.


No data available for NIFTY PVT BANK (NIFTYPVTBANK.NS)
Processing NIFTY PSE (^CNXPSE)...
Waiting 3.3 seconds before requesting data...
Processing NIFTY ENERGY (^CNXENERGY)...
Waiting 3.4 seconds before requesting data...
Processing NIFTY FARMA (^CNXPHARMA)...
Waiting 4.1 seconds before requesting data...
Processing NIFTY MNC (^CNXMNC)...
Waiting 4.3 seconds before requesting data...
Processing NIFTY INFRA (^CNXINFRA)...
Waiting 5.2 seconds before requesting data...

Analysis complete. Results saved to index_analysis\results\indian_indices_analysis_20250227_160915.xlsx


In [ ]:
alpha vantage api key=EM154MX6UKVK7NJV

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import time
import os
import json
import random

class NSEIndexAnalyzer:
    def __init__(self):
        self.base_dir = 'index_analysis'
        self.results_dir = os.path.join(self.base_dir, 'results')
        os.makedirs(self.results_dir, exist_ok=True)
        
        # List of Indian indices to analyze with their Yahoo Finance tickers
        # Added alternative tickers for problematic indices
        self.equity_market_indices = [
            {
                'name': 'NIFTY FIN SERVICES',
                'ticker': '^CNXFIN',
                'alt_ticker': 'NIFTY_FIN_SERVICE.NS'  # Alternative ticker for Financial Services
            },
            {
                'name': 'NIFTY FMCG',
                'ticker': '^CNXFMCG',
                'alt_ticker': 'NIFTYFMCG.NS'
            },
            {
                'name': 'NIFTY METAL',
                'ticker': '^CNXMETAL',
                'alt_ticker': 'NIFTYMETAL.NS'
            },
            {
                'name': 'NIFTY REALTY',
                'ticker': '^CNXREALTY',
                'alt_ticker': 'NIFTYREALTY.NS'
            },
            {
                'name': 'NIFTY MEDIA',
                'ticker': '^CNXMEDIA',
                'alt_ticker': 'NIFTYMEDIA.NS'
            },
            {
                'name': 'NIFTY PSU BANK',
                'ticker': '^CNXPSUBANK',
                'alt_ticker': 'NIFTYPSUBANK.NS'
            },
            {
                'name': 'NIFTY MIDCAP 50',
                'ticker': '^NSEMDCP50',
                'alt_ticker': 'NIFTYMIDC50.NS'
            },
            {
                'name': 'NIFTY SMALLCAP 50',
                'ticker': 'NIFTYSMLCAP50.NS',
                'alt_ticker': 'NIFTYSMALL50.NS'
            },
            {
                'name': 'NIFTY PSE',
                'ticker': '^CNXPSE',
                'alt_ticker': 'NIFTYPSE.NS'
            },
            {
                'name': 'NIFTY ENERGY',
                'ticker': '^CNXENERGY',
                'alt_ticker': 'NIFTYENERGY.NS'
            },
            {
                'name': 'NIFTY MNC',
                'ticker': '^CNXMNC',
                'alt_ticker': 'NIFTYMNC.NS'
            },
            {
                'name': 'NIFTY INFRA',
                'ticker': '^CNXINFRA',
                'alt_ticker': 'NIFTYINFRA.NS'
            }
        ]

    def calculate_rsi(self, prices_series, periods=14):
        """RSI calculation method - improved to handle edge cases"""
        try:
            # Convert to pandas Series if it's not already
            if not isinstance(prices_series, pd.Series):
                prices_series = pd.Series(prices_series)
            
            # Drop NaN values to ensure clean calculations
            prices_series = prices_series.dropna()
            
            if len(prices_series) < periods + 1:
                print(f"Not enough data points for RSI calculation. Need {periods+1}, got {len(prices_series)}")
                return None
                
            # Calculate price changes
            delta = prices_series.diff().dropna()
            
            # Split gains and losses
            gains = delta.copy()
            losses = delta.copy()
            
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calculate initial averages
            if len(gains) < periods:
                print(f"Not enough data points after diff for RSI calculation")
                return None
                
            first_avg_gain = gains[:periods].mean()
            first_avg_loss = losses[:periods].mean()
            
            # Check for zero division
            if first_avg_loss == 0:
                if first_avg_gain == 0:
                    # Both are zero, RSI is 50 (neutral)
                    return 50.0
                else:
                    # Only losses are zero, RSI is 100 (all gains)
                    return 100.0
            
            # First RSI value
            rs = first_avg_gain / first_avg_loss
            rsi = 100.0 - (100.0 / (1.0 + rs))
            
            # Calculate subsequent averages using smoothing method
            for i in range(periods, len(delta)):
                avg_gain = (first_avg_gain * (periods - 1) + gains.iloc[i]) / periods
                avg_loss = (first_avg_loss * (periods - 1) + losses.iloc[i]) / periods
                
                first_avg_gain = avg_gain
                first_avg_loss = avg_loss
                
                # Avoid division by zero
                if avg_loss == 0:
                    rsi = 100.0
                else:
                    rs = avg_gain / avg_loss
                    rsi = 100.0 - (100.0 / (1.0 + rs))
            
            return float(rsi)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def calculate_tema(self, prices, period=50):
        """
        Calculate Triple Exponential Moving Average (TEMA) - improved
        
        Args:
            prices (pd.Series): Price series
            period (int): Period for TEMA calculation
        
        Returns:
            float: TEMA value
        """
        try:
            # Convert to pandas Series if it's not already
            if not isinstance(prices, pd.Series):
                prices = pd.Series(prices)
            
            # Drop NaN values to ensure clean calculations
            prices = prices.dropna()
            
            if len(prices) < period:
                print(f"Not enough data points for TEMA calculation. Need {period}, got {len(prices)}")
                return None
            
            # Calculate EMAs
            ema1 = prices.ewm(span=period, adjust=False).mean()
            ema2 = ema1.ewm(span=period, adjust=False).mean()
            ema3 = ema2.ewm(span=period, adjust=False).mean()
            
            # TEMA Formula: TEMA = 3 * EMA1 - 3 * EMA2 + EMA3
            tema = 3 * ema1.iloc[-1] - 3 * ema2.iloc[-1] + ema3.iloc[-1]
            
            return float(tema)
        
        except Exception as e:
            print(f"Error calculating TEMA: {e}")
            return None

    def get_index_data(self, index_info, max_retries=3):
        """Fetch detailed index data with retry logic and alternative tickers"""
        # Try with primary ticker first
        ticker_symbol = index_info['ticker']
        for attempt in range(max_retries):
            try:
                # Add randomized delay between 3-6 seconds to avoid rate limiting
                delay = 3 + random.random() * 3
                print(f"Waiting {delay:.1f} seconds before requesting data for {ticker_symbol}...")
                time.sleep(delay)
                
                ticker = yf.Ticker(ticker_symbol)
                
                # Get historical data for 1 year
                hist_data = ticker.history(period="1y", interval="1d")

                # Check if we have enough data
                if len(hist_data) < 50:
                    print(f"Insufficient data for {index_info['name']} using {ticker_symbol}: only {len(hist_data)} days")
                    print(f"Trying alternative ticker...")
                    
                    # If primary ticker doesn't work, try alternative ticker
                    alt_ticker = index_info.get('alt_ticker')
                    if not alt_ticker:
                        print(f"No alternative ticker available for {index_info['name']}")
                        return None
                        
                    time.sleep(2)  # Wait before trying alternative
                    ticker = yf.Ticker(alt_ticker)
                    hist_data = ticker.history(period="1y", interval="1d")
                    
                    if len(hist_data) < 50:
                        print(f"Insufficient data for {index_info['name']} using alternative {alt_ticker}: only {len(hist_data)} days")
                        return None
                    else:
                        print(f"Successfully retrieved data using alternative ticker {alt_ticker}")
                        ticker_symbol = alt_ticker  # Update ticker symbol for output

                # Check for NaN values in Close prices
                if hist_data['Close'].isna().any():
                    print(f"Warning: {index_info['name']} has NaN values in Close prices, attempting to handle them")
                    # Fill NaN with previous values or drop them
                    hist_data['Close'] = hist_data['Close'].fillna(method='ffill')
                
                # Calculate basic metrics
                latest_price = hist_data['Close'].iloc[-1]
                
                # Calculate indicators
                prices_series = hist_data['Close']
                latest_rsi = self.calculate_rsi(prices_series)
                
                # Calculate EMA-50 and TEMA-50
                ema_50 = prices_series.ewm(span=50, adjust=False).mean().iloc[-1]
                tema_50 = self.calculate_tema(prices_series)
                
                # Calculate 20-day and 50-day simple moving averages
                sma_20 = hist_data['Close'].rolling(window=20).mean().iloc[-1] if len(hist_data) >= 20 else None
                sma_50 = hist_data['Close'].rolling(window=50).mean().iloc[-1] if len(hist_data) >= 50 else None
                
                # Calculate daily returns
                daily_returns = hist_data['Close'].pct_change()
                
                # Calculate volatility (20-day standard deviation of returns)
                volatility_20 = daily_returns.rolling(window=20).std().iloc[-1] * (252 ** 0.5)  # Annualized
                
                # Calculate 52-week high and low
                high_52w = hist_data['High'].max()
                low_52w = hist_data['Low'].min()
                
                # Calculate percent from 52-week high
                pct_from_high = (latest_price - high_52w) / high_52w * 100
                
                # Prepare index information dictionary
                index_data = {
                    'Index_Name': index_info['name'],
                    'Ticker': ticker_symbol,
                    'Latest_Price': round(latest_price, 2),
                    'RSI_14': round(latest_rsi, 2) if latest_rsi is not None else 'N/A',
                    'EMA_50': round(ema_50, 2) if ema_50 is not None else 'N/A',
                    'TEMA_50': round(tema_50, 2) if tema_50 is not None else 'N/A',
                    'TEMA_50-EMA_50': round(tema_50 - ema_50, 2) if tema_50 is not None and ema_50 is not None else 'N/A',
                    'SMA_20': round(sma_20, 2) if sma_20 is not None else 'N/A',
                    'SMA_50': round(sma_50, 2) if sma_50 is not None else 'N/A',
                    'Volatility_20D': round(volatility_20 * 100, 2) if not pd.isna(volatility_20) else 'N/A',
                    '52W_High': round(high_52w, 2),
                    '52W_Low': round(low_52w, 2),
                    'Pct_From_52W_High': round(pct_from_high, 2),
                    'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 'N/A',
                    'Daily_Return': round(daily_returns.iloc[-1] * 100, 2) if not pd.isna(daily_returns.iloc[-1]) else 'N/A'
                }
                
                return index_data

            except Exception as e:
                error_msg = str(e)
                print(f"Attempt {attempt+1}/{max_retries} - Error processing {index_info['name']} ({ticker_symbol}): {error_msg}")
                
                if "Too Many Requests" in error_msg or "Rate limited" in error_msg:
                    # Exponential backoff - wait longer with each retry
                    wait_time = (2 ** attempt) * 10 + random.random() * 5
                    print(f"Rate limited. Waiting {wait_time:.1f} seconds before retrying...")
                    time.sleep(wait_time)
                else:
                    # For other errors, wait a shorter time
                    time.sleep(2)
                    
                # If we had an error with the primary ticker on the last attempt, try the alternative
                if attempt == max_retries - 1:
                    alt_ticker = index_info.get('alt_ticker')
                    if alt_ticker and alt_ticker != ticker_symbol:
                        print(f"Trying alternative ticker {alt_ticker} after failed attempts with primary ticker")
                        ticker_symbol = alt_ticker
                        attempt = 0  # Reset attempts for the new ticker
        
        print(f"Failed to retrieve data for {index_info['name']} after all attempts")
        return None

    def analyze_indices(self):
        """Analyze all indices and generate Excel report"""
        # Timestamp for unique filename
        current_date = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_file = os.path.join(self.results_dir, f'indian_indices_analysis_{current_date}.xlsx')
        
        # Process all indices
        results = []
        for index_info in self.equity_market_indices:
            print(f"Processing {index_info['name']} ({index_info['ticker']})...")
            index_data = self.get_index_data(index_info)
            if index_data:
                results.append(index_data)
                print(f"Successfully processed {index_info['name']}")
            else:
                print(f"Skipping {index_info['name']} due to data retrieval issues")
        
        # Convert to DataFrame
        if results:
            df = pd.DataFrame(results)
            
            # Create Excel writer
            with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                # Write main index data
                df.to_excel(writer, sheet_name='Indian_Indices', index=False)
                
                # Get workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets['Indian_Indices']
                
                # Add formats for conditional formatting
                green_format = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
                red_format = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
                
                # Apply conditional formatting to RSI column
                rsi_col = df.columns.get_loc('RSI_14') + 1  # +1 for Excel's 1-indexing
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'less than', 'value': 30, 'format': green_format})
                worksheet.conditional_format(1, rsi_col, len(results), rsi_col, 
                                          {'type': 'cell', 'criteria': 'greater than', 'value': 70, 'format': red_format})
            
            print(f"\nAnalysis complete. Results saved to {excel_file}")
            return df  # Return the dataframe for further analysis if needed
        else:
            print("No index data was successfully retrieved.")
            return None

if __name__ == "__main__":
    analyzer = NSEIndexAnalyzer()
    analyzer.analyze_indices()

Processing NIFTY FIN SERVICES (^CNXFIN)...
Waiting 3.3 seconds before requesting data for ^CNXFIN...
Insufficient data for NIFTY FIN SERVICES using ^CNXFIN: only 1 days
Trying alternative ticker...
Successfully retrieved data using alternative ticker NIFTY_FIN_SERVICE.NS
Successfully processed NIFTY FIN SERVICES
Processing NIFTY FMCG (^CNXFMCG)...
Waiting 4.8 seconds before requesting data for ^CNXFMCG...
Successfully processed NIFTY FMCG
Processing NIFTY METAL (^CNXMETAL)...
Waiting 5.9 seconds before requesting data for ^CNXMETAL...
Successfully processed NIFTY METAL
Processing NIFTY REALTY (^CNXREALTY)...
Waiting 5.9 seconds before requesting data for ^CNXREALTY...
Successfully processed NIFTY REALTY
Processing NIFTY MEDIA (^CNXMEDIA)...
Waiting 5.3 seconds before requesting data for ^CNXMEDIA...
Successfully processed NIFTY MEDIA
Processing NIFTY PSU BANK (^CNXPSUBANK)...
Waiting 5.2 seconds before requesting data for ^CNXPSUBANK...
Successfully processed NIFTY PSU BANK
Processing 

$NIFTYSMALL50.NS: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


Insufficient data for NIFTY SMALLCAP 50 using alternative NIFTYSMALL50.NS: only 0 days
Skipping NIFTY SMALLCAP 50 due to data retrieval issues
Processing NIFTY PSE (^CNXPSE)...
Waiting 5.7 seconds before requesting data for ^CNXPSE...
Successfully processed NIFTY PSE
Processing NIFTY ENERGY (^CNXENERGY)...
Waiting 3.2 seconds before requesting data for ^CNXENERGY...
Successfully processed NIFTY ENERGY
Processing NIFTY MNC (^CNXMNC)...
Waiting 5.2 seconds before requesting data for ^CNXMNC...
Successfully processed NIFTY MNC
Processing NIFTY INFRA (^CNXINFRA)...
Waiting 3.9 seconds before requesting data for ^CNXINFRA...
Successfully processed NIFTY INFRA

Analysis complete. Results saved to index_analysis\results\indian_indices_analysis_20250227_164540.xlsx
